<a href="https://colab.research.google.com/github/ipinmi/grounded-vqa-fm/blob/detection/clip_vqa_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!unzip vcr1annots.zip

Archive:  vcr1annots.zip
   creating: vcr1annots/
  inflating: vcr1annots/train.jsonl  
  inflating: vcr1annots/cocoontology.json  
  inflating: vcr1annots/val.jsonl    


In [3]:
!unzip /content/drive/MyDrive/vcr1images.zip

Streaming output truncated to the last 5000 lines.
  inflating: vcr1images/lsmdc_0041_The_Sixth_Sense/0041_The_Sixth_Sense_00.06.32.220-00.06.33.151@0.json  
  inflating: __MACOSX/vcr1images/lsmdc_0041_The_Sixth_Sense/._0041_The_Sixth_Sense_00.06.32.220-00.06.33.151@0.json  
  inflating: vcr1images/lsmdc_0041_The_Sixth_Sense/0041_The_Sixth_Sense_00.57.25.291-00.57.29.919@0.jpg  
  inflating: __MACOSX/vcr1images/lsmdc_0041_The_Sixth_Sense/._0041_The_Sixth_Sense_00.57.25.291-00.57.29.919@0.jpg  
  inflating: vcr1images/lsmdc_0041_The_Sixth_Sense/0041_The_Sixth_Sense_00.19.34.739-00.19.41.016@2.json  
  inflating: __MACOSX/vcr1images/lsmdc_0041_The_Sixth_Sense/._0041_The_Sixth_Sense_00.19.34.739-00.19.41.016@2.json  
  inflating: vcr1images/lsmdc_0041_The_Sixth_Sense/0041_The_Sixth_Sense_01.23.23.395-01.23.29.821@0.jpg  
  inflating: __MACOSX/vcr1images/lsmdc_0041_The_Sixth_Sense/._0041_The_Sixth_Sense_01.23.23.395-01.23.29.821@0.jpg  
  inflating: vcr1images/lsmdc_0041_The_Sixth_Sense/00

In [4]:
!git clone

Cloning into 'grounded-vqa-fm'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 53 (delta 18), reused 29 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (53/53), 104.41 KiB | 848.00 KiB/s, done.
Resolving deltas: 100% (18/18), done.


In [5]:
!cd grounded-vqa-fm && git checkout detection

Branch 'detection' set up to track remote branch 'detection' from 'origin'.
Switched to a new branch 'detection'


In [6]:
from huggingface_hub import notebook_login

In [7]:
notebook_login()

In [8]:
import os
os.chdir('/content/grounded-vqa-fm')

In [9]:
import argparse
import matplotlib.pyplot as plt
import json
import subprocess


##MODULES
from vqa_interface.clip_interface import run_CLIP_batch, eval_on_accuracy
from vcr_data.vcr_dataloader import (
    VCRDataExtractor,
    VCRDataset,
    VCRDataLoader,
    BatchSampler,
)

In [10]:
VCR_ANNOTS_DIR = "/content/vcr1annots"
VCR_IMAGES_DIR = "/content/vcr1images"
results_path = "/content/results"

subprocess.run(["mkdir", "-p", results_path])

CompletedProcess(args=['mkdir', '-p', '/content/results'], returncode=0)

In [11]:
def save_json(data, path):
    with open(path, "w") as f:
        json.dump(data, f)


def main():
    extracted_vcr = VCRDataExtractor(
        VCR_ANNOTS_DIR,
        VCR_IMAGES_DIR,
        mode="answer",
        split="val",
        only_use_relevant_dets=False,
    )
    dataset = VCRDataset(extracted_vcr, "vqa")
    batch_sampler = BatchSampler(dataset, batch_size=4)
    dataloader = VCRDataLoader(dataset, batch_sampler=batch_sampler)

    # Run the CLIP model
    vqa_results = run_CLIP_batch(dataloader)
    save_json(vqa_results, "/content/results/clip_vqa_results.json")
    # print(vqa_results)

    # Evaluate the model
    accuracy, pred_value_1, total = eval_on_accuracy(vqa_results)
    print(f"Accuracy: {accuracy}")
    print(f"No of times Predicted value is 1: {pred_value_1}")
    print(f"Total: {total}")




In [12]:
main()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

 11%|█▏        | 3001/26534 [21:21<2:47:29,  2.34it/s]


Accuracy: 0.25316455696202533
No of times Predicted value is 1: 3002
Total: 3002
